In [2]:
# !unzip klimb_llm_optimization_challenge.zip

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [4]:
train_dir = "klimb_llm_optimization_challenge/seg_train"
test_dir = "klimb_llm_optimization_challenge/seg_test"

In [5]:
batch_size = 32
img_height = 150
img_width = 150

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
	train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 14034 files belonging to 6 classes.
Using 11228 files for training.


In [7]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3000 files belonging to 6 classes.
Using 600 files for validation.


In [8]:
base_model = keras.applications.ResNet152(
		weights='imagenet',  # Load weights pre-trained on ImageNet.
		input_shape=(img_height, img_width, 3),
		include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False
inputs = keras.Input(shape=(img_height, img_width, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

234698864/234698864 [==============================] - 1s 0us/step


In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 5, 5, 2048)        58370944  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 12294     
                                                                 
Total params: 58383238 (222.71 MB)
Trainable params: 12294 (48.02 KB)
Non-trainable params: 58370944 (222.67 MB)
_________________________________________________________________


In [10]:
model.compile(
		optimizer=keras.optimizers.Adam(),
		loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
		metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [11]:
epochs = 20
model.fit(train_ds, epochs=epochs)

Epoch 1/20
351/351 [==============================] - 59s 135ms/step - loss: 0.4092 - sparse_categorical_accuracy: 0.8578
Epoch 2/20
351/351 [==============================] - 50s 141ms/step - loss: 0.2517 - sparse_categorical_accuracy: 0.9113
Epoch 3/20
351/351 [==============================] - 49s 140ms/step - loss: 0.2147 - sparse_categorical_accuracy: 0.9274
Epoch 4/20
351/351 [==============================] - 51s 143ms/step - loss: 0.1832 - sparse_categorical_accuracy: 0.9340
Epoch 5/20
351/351 [==============================] - 49s 139ms/step - loss: 0.1609 - sparse_categorical_accuracy: 0.9428
Epoch 6/20
351/351 [==============================] - 50s 142ms/step - loss: 0.1450 - sparse_categorical_accuracy: 0.9479
Epoch 7/20
351/351 [==============================] - 50s 143ms/step - loss: 0.1292 - sparse_categorical_accuracy: 0.9551
Epoch 8/20
351/351 [==============================] - 50s 143ms/step - loss: 0.1181 - sparse_categorical_accuracy: 0.9605
Epoch 9/20
351/351 [====

In [12]:
results = model.evaluate(test_ds)
print(f"Test accuracy with trained teacher model:{results[1]*100 :.2f} %")

19/19 [==============================] - 7s 212ms/step - loss: 0.3816 - sparse_categorical_accuracy: 0.8933
Test accuracy with trained teacher model:89.33 %


In [31]:
# Define Student model
inputs_student = keras.Input(shape=(img_height, img_width, 3))
x_student = layers.Conv2D(32, (3, 3), activation='relu')(inputs_student)
x_student = layers.MaxPooling2D((2, 2))(x_student)
x_student = layers.Conv2D(64, (3, 3), activation='relu')(x_student)
x_student = layers.MaxPooling2D((2, 2))(x_student)
x_student = layers.GlobalAveragePooling2D()(x_student)
outputs_student = layers.Dense(6)(x_student)
student_model = keras.Model(inputs_student, outputs_student)

In [32]:
student_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 global_average_pooling2d_4  (None, 64)                0   

In [33]:
#Transfer weights from master model to student model
for master_layer, student_layer in zip(model.layers, student_model.layers):
    if isinstance(master_layer, tf.keras.layers.Conv2D) and isinstance(student_layer, tf.keras.layers.Conv2D):
        student_layer.set_weights(master_layer.get_weights())

In [34]:
student_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


In [40]:
epochs_student = 50
student_model.fit(train_ds, epochs=epochs_student)

Epoch 1/50
351/351 [==============================] - 8s 24ms/step - loss: 0.0781 - sparse_categorical_accuracy: 0.9748
Epoch 2/50
351/351 [==============================] - 7s 18ms/step - loss: 0.0673 - sparse_categorical_accuracy: 0.9771
Epoch 3/50
351/351 [==============================] - 8s 23ms/step - loss: 0.0468 - sparse_categorical_accuracy: 0.9859
Epoch 4/50
351/351 [==============================] - 7s 19ms/step - loss: 0.0530 - sparse_categorical_accuracy: 0.9835
Epoch 5/50
351/351 [==============================] - 9s 24ms/step - loss: 0.0611 - sparse_categorical_accuracy: 0.9782
Epoch 6/50
351/351 [==============================] - 7s 19ms/step - loss: 0.0780 - sparse_categorical_accuracy: 0.9751
Epoch 7/50
351/351 [==============================] - 9s 25ms/step - loss: 0.0649 - sparse_categorical_accuracy: 0.9783
Epoch 8/50
351/351 [==============================] - 7s 19ms/step - loss: 0.0628 - sparse_categorical_accuracy: 0.9797
Epoch 9/50
351/351 [====================

In [42]:
results_master = student_model.evaluate(test_ds)
print(f"Test accuracy with trained teacher model: {results_master[1]*100:.2f} %")

19/19 [==============================] - 1s 23ms/step - loss: 1.1519 - sparse_categorical_accuracy: 0.8533
Test accuracy with trained teacher model: 85.33 %


In [43]:
# Calculate model size and parameter ratios
master_model_size = len(model.to_json())
student_model_size = len(student_model.to_json())
model_size_ratio = master_model_size / student_model_size

master_model_params = model.count_params()
student_model_params = student_model.count_params()
param_ratio = master_model_params / student_model_params

In [44]:
# Display ratios
print(f"Model Size Ratio (Master/Student): {model_size_ratio:.2f}")
print(f"Parameter Ratio (Master/Student): {param_ratio:.2f}")

Model Size Ratio (Master/Student): 89.30
Parameter Ratio (Master/Student): 2951.33


In [45]:
import time

# Function to measure latency for model predictions
def measure_latency(model, dataset):
    start_time = time.time()

    for images, labels in dataset:
        _ = model.predict(images)  # Assuming a single prediction for simplicity

    end_time = time.time()
    latency = (end_time - start_time) / len(dataset)

    return latency


In [46]:
# Measure latency for the master model
master_latency = measure_latency(model, test_ds)
print(f"Latency for Master Model: {master_latency:.4f} seconds per prediction")

# Measure latency for the student model
student_latency = measure_latency(student_model, test_ds)
print(f"Latency for Student Model: {student_latency:.4f} seconds per prediction")

1/1 [==============================] - 3s 3s/step
Latency for Master Model: 0.5462 seconds per prediction
1/1 [==============================] - 0s 56ms/step
Latency for Student Model: 0.0756 seconds per prediction


In [ ]:
# End-to-End Functionality: The entire pipeline, from the model building to the final prediction
# from both master and student models should be operational without any errors.
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Function to load data
def load_data(train_dir, test_dir, img_height, img_width, batch_size):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    return train_ds, test_ds

# Define the master model
def build_master_model(img_height, img_width):
    base_model = keras.applications.ResNet152(
        weights='imagenet',
        input_shape=(img_height, img_width, 3),
        include_top=False
    )
    base_model.trainable = False

    inputs_master = keras.Input(shape=(img_height, img_width, 3))
    x_master = base_model(inputs_master, training=False)
    x_master = keras.layers.GlobalAveragePooling2D()(x_master)
    outputs_master = keras.layers.Dense(6)(x_master)
    master_model = keras.Model(inputs_master, outputs_master)

    return master_model

# Define the student model
def build_student_model(img_height, img_width):
    inputs_student = keras.Input(shape=(img_height, img_width, 3))
    x_student = layers.Conv2D(32, (3, 3), activation='relu')(inputs_student)
    x_student = layers.MaxPooling2D((2, 2))(x_student)
    x_student = layers.Conv2D(64, (3, 3), activation='relu')(x_student)
    x_student = layers.MaxPooling2D((2, 2))(x_student)
    x_student = layers.GlobalAveragePooling2D()(x_student)
    outputs_student = layers.Dense(6)(x_student)
    student_model = keras.Model(inputs_student, outputs_student)

    return student_model

# Function to transfer weights from master to student model
def transfer_weights(master_model, student_model):
    for master_layer, student_layer in zip(master_model.layers, student_model.layers):
        if isinstance(master_layer, tf.keras.layers.Conv2D) and isinstance(student_layer, tf.keras.layers.Conv2D):
            student_layer.set_weights(master_layer.get_weights())

# Function to train and evaluate models
def train_and_evaluate_models(train_ds, test_ds, master_model, student_model, epochs):
    # Compile master model
    master_model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    # Train master model
    master_model.fit(train_ds, epochs=epochs)

    # Compile student model
    student_model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    # Transfer weights from master to student model
    transfer_weights(master_model, student_model)

    # Train student model
    student_model.fit(train_ds, epochs=epochs)

    # Evaluate models on test data
    results_master = master_model.evaluate(test_ds)
    print(f"Test accuracy with trained master model: {results_master[1]*100:.2f} %")

    results_student = student_model.evaluate(test_ds)
    print(f"Test accuracy with trained student model: {results_student[1]*100:.2f} %")

# Set up parameters
img_height = 150
img_width = 150
batch_size = 32
epochs = 10
train_dir = "klimb_llm_optimization_challenge/seg_train"
test_dir = "klimb_llm_optimization_challenge/seg_test"

# Load data
train_ds, test_ds = load_data(train_dir, test_dir, img_height, img_width, batch_size)

# Build models
master_model = build_master_model(img_height, img_width)
student_model = build_student_model(img_height, img_width)

# Train and evaluate models
train_and_evaluate_models(train_ds, test_ds, master_model, student_model, epochs)
